In [93]:
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
import tensorflow as tf
import pathlib
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator


tf.__version__

'2.4.0'

In [94]:
batch_size = 32
img_height = 180
img_width = 180 
data_dir = pathlib.Path('vision/training_set') 
datatest_dir = pathlib.Path('vision/test_set')
print(data_dir)
AUTOTUNE = tf.data.AUTOTUNE

vision\training_set


In [95]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1284 files belonging to 3 classes.
Using 771 files for training.


In [96]:
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=0.4,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 1284 files belonging to 3 classes.
Using 513 files for validation.


In [97]:
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [98]:
num_classes = 3

model = Sequential([
  layers.experimental.preprocessing.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])


In [108]:

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling_6 (Rescaling)      (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 180, 180, 16)      448       
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 90, 90, 16)        0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 90, 90, 32)        4640      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 45, 45, 32)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 45, 45, 64)        18496     
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 22, 22, 64)       

In [99]:
class_names = train_ds.class_names
print(class_names)

['burn_disease', 'healthy', 'leafspot']


In [100]:
for image_batch, labels_batch in train_ds:
    print(image_batch.shape)
    print(labels_batch.shape)
    break

(32, 180, 180, 3)
(32,)


In [101]:
model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [102]:
model.fit(train_ds, validation_data=val_ds, epochs=50)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
24/25 [===========================>..] - ETA: 0s - loss: 1.1315 - accuracy: 0.5679WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x00000267B6C4A0D0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) 

In [103]:
 rootPath = './vision/'

In [104]:
 imageGenerator = ImageDataGenerator(
    
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.2,
        zoom_range=0.1,
        validation_split=.1,
        horizontal_flip=True,
        vertical_flip=True, 
        fill_mode='nearest')

In [105]:
testGen = imageGenerator.flow_from_directory(
    os.path.join(rootPath, 'test_set'),
    target_size=(180, 180),
)

Found 15 images belonging to 3 classes.


In [106]:
pred = model.predict(testGen)
img_list = testGen.filenames
pred_list = []
for i in range(len(pred)):
    pred_list.append(np.argmax(pred[i]))

df = pd.DataFrame(list(zip(img_list, pred_list)), columns =['Name', 'pred'])
print(df)

df.to_csv('submission.csv', index=False)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
                      Name  pred
0   burn_disease\burn1.png     0
1   burn_disease\burn2.png     0
2   burn_disease\burn3.png     0
3   burn_disease\burn4.png     0
4   burn_disease\burn5.png     1
5         healthy\hel1.jpg     2
6         healthy\hel2.jpg     0
7         healthy\hel3.jpg     2
8         healthy\hel4.jpg     2
9         healthy\hel5.jpg     2
10  leafspot\le

In [107]:
df

,Name,pred
0,burn_disease\burn1.png,0
1,burn_disease\burn2.png,0
2,burn_disease\burn3.png,0
3,burn_disease\burn4.png,0
4,burn_disease\burn5.png,1
5,healthy\hel1.jpg,2
6,healthy\hel2.jpg,0
7,healthy\hel3.jpg,2
8,healthy\hel4.jpg,2
9,healthy\hel5.jpg,2
